# Faulty Takata Airbags using Logistic Regression

**The story:**
- https://www.nytimes.com/2014/09/12/business/air-bag-flaw-long-known-led-to-recalls.html
- https://www.nytimes.com/2014/11/07/business/airbag-maker-takata-is-said-to-have-conducted-secret-tests.html
- https://www.nytimes.com/interactive/2015/06/22/business/international/takata-airbag-recall-list.html
- https://www.nytimes.com/2016/08/27/business/takata-airbag-recall-crisis.html

This story, done by The New York Times, investigates the content in complaints made to National Highway Traffic Safety Administration (NHTSA) by customers who had bad experiences with Takata airbags in their cars. Eventually, car companies had to recall airbags made by the airbag supplier that promised a cheaper alternative. 

**Author:** Daeil Kim did a more complex version of this particular analysis - [presentation here](https://www.slideshare.net/mortardata/daeil-kim-at-the-nyc-data-science-meetup)

**Topics:** Decision Trees, Random Forests

**Datasets**

* **sampled-labeled.csv:** a sample of vehicle complaints, labeled with being suspicious or not

## What's the goal?

It was too much work to read twenty years of vehicle comments to find the ones related to dangerous airbags! Because we're lazy, we wanted the computer to do this for us. We did this before with a classifier that used logistic regression, now we're going to try a different one.

# Our code

## Setup

In [2]:
import pandas as pd
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import train_test_split

# Allow us to display 100 columns at a time, and 100 characters in each column (instead of ...)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 100)

## Read in our labeled data

We aren't going to be using the unlabeled dataset this time, we're only going to look at **how our classifier works.** We'll start by reading in our complaints that have labeled attached to them.

**Read in `sampled-labeled.csv` and check how many suspicious/not suspicious complaints we have.**

In [2]:
labeled = pd.read_csv("data/sampled-labeled.csv")
labeled.head()

,is_suspicious,CDESCR
0,0.0,"ALTHOUGH I LOVED THE CAR OVERALL AT THE TIME I DECIDED TO OWN, , MY DREAM CAR CADILLAC CTS HAS T..."
1,0.0,"CONSUMER SHUT SLIDING DOOR WHEN ALL POWER LOCKS ON ALL DOORS LOCKED BY ITSELF, TRAPPING INFANT I..."
2,0.0,DRIVERS SEAT BACK COLLAPSED AND BENT WHEN REAR ENDED. PLEASE DESCRIBE DETAILS. TT
3,0.0,TL* THE CONTACT OWNS A 2009 NISSAN ALTIMA. THE CONTACT STATED THAT THE START BUTTON FOR THE IGNI...
4,0.0,THE FRONT MIDDLE SEAT DOESN'T LOCK IN PLACE. *AK


In [3]:
labeled.is_suspicious.value_counts()

0.0    150
1.0     15
Name: is_suspicious, dtype: int64

150 non-suspicious and 15 suspicious is a pretty terrible ratio, but we're remarkably lazy and not very many of the comments are actually suspicious.

Now that we've read a few, let's train our classifier

## Creating features

### Selecting our features and building a features dataframe

Last time, we can thought of some words or phrases that might make a comment interesting or not interesting. We came up with this list:

* airbag
* air bag
* failed
* did not deploy
* violent
* explode
* shrapnel

These **features** are the things that the machine learning algorithm is going to look for when it's reading. There are lots of words in each complaint, but these are the only ones we'll tell the classifier to pay attention to!

To determine if a word is in `CDESCR`, we can use `.str.contains`. Because computers only like numbers, though, we need to use `.astype(int)` to change it from `True`/`False` to `1`/`0`. 

In [4]:
train_df = pd.DataFrame({
    'is_suspicious': labeled.is_suspicious,
    'airbag': labeled.CDESCR.str.contains("AIRBAG", na=False).astype(int),
    'air bag': labeled.CDESCR.str.contains("AIR BAG", na=False).astype(int),
    'failed': labeled.CDESCR.str.contains("FAILED", na=False).astype(int),
    'did not deploy': labeled.CDESCR.str.contains("DID NOT DEPLOY", na=False).astype(int),
    'violent': labeled.CDESCR.str.contains("VIOLENT", na=False).astype(int),
    'explode': labeled.CDESCR.str.contains("EXPLODE", na=False).astype(int),
    'shrapnel': labeled.CDESCR.str.contains("SHRAPNEL", na=False).astype(int),
})
train_df.head()

,is_suspicious,airbag,air bag,failed,did not deploy,violent,explode,shrapnel
0,0.0,0,0,0,0,0,0,0
1,0.0,0,0,0,0,0,0,0
2,0.0,0,0,0,0,0,0,0
3,0.0,0,0,0,0,0,0,0
4,0.0,0,0,0,0,0,0,0


Let's see how big our dataset is, and then remove any rows that are missing data (not all of them are labeled).

In [5]:
train_df.shape

(350, 8)

In [6]:
train_df = train_df.dropna()
train_df.shape

(165, 8)

## Creating our classifier

Any time you're bulding a classifier, doing regression, or most anything with machine learning, you're using a **model**. It **models** the relationship between the inputs and the outputs.

### Recap: Classification with Logistic Regression

Last time we used a classifier based on **Logistic Regression**. First we split into `X` (our features) and `y` (our labels), and trained the classifier on them.

In [7]:
from sklearn.linear_model import LogisticRegression

X = train_df.drop(columns='is_suspicious')
y = train_df.is_suspicious

clf = LogisticRegression(C=1e9, solver='lbfgs')

clf.fit(X, y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

After we built our classifier, we tested it and found it didn't work very well.

In [8]:
from sklearn.metrics import confusion_matrix

y_true = y
y_pred = clf.predict(X)

matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['not suspicious', 'suspicious'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted not suspicious,Predicted suspicious
Is not suspicious,150,0
Is suspicious,13,2


In [3]:
metrics.confusion_matrix(y_test,y_pred)

NameError: name 'y_test' is not defined

To understand a logisic regression classifier, we looked at the coefficients and the odds ratios.

In [9]:
import numpy as np

feature_names = X.columns
coefficients = clf.coef_[0]

pd.DataFrame({
    'feature': feature_names,
    'coefficient (log odds ratio)': coefficients,
    'odds ratio': np.exp(coefficients).round(4)
}).sort_values(by='odds ratio', ascending=False)

,feature,coefficient (log odds ratio),odds ratio
4,violent,41.423096,9.768364e+17
5,explode,1.269048,3.557500e+00
1,air bag,1.268123,3.554200e+00
0,airbag,0.945612,2.574400e+00
2,failed,-27.175214,0.000000e+00
3,did not deploy,-37.906428,0.000000e+00
6,shrapnel,-13.204894,0.000000e+00


### Classification with Decision Trees

We can also use a classifier called a **decision tree**. All you need to do is have one new import and change the line where you create your classifier.

In [4]:
#from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier



Confusion matrix code looks exactly the same.

In [5]:
from sklearn.metrics import confusion_matrix



When using a decision tree, **using the classifier is the same, but the code to understand the classifier is a bit different.** Instead of coefficients, we're going to look at **feature importance.**

The most fun part of using a decision tree is **visualizing it.**

And the best part is: almost everything you can do with a logistic regression classifier you can do with a decision tree. Most of the time you can just **change your classifier to see if it does better.**

Decision trees also have a lot of simple options.

### A random forest is usually even better

Although in this case our inputs are terrible so it's still not very good. Garbage in, garbage out.

We'l change our classifier to be

```python
clf = RandomForestClassifier(n_estimators=100)
```

and it will use 100 decision **trees** to make a **forest**.